In [ ]:
import pandas as pd
import datetime as dt
import time
import io
import gzip
import os
import logging
from dotenv import load_dotenv
from concurrent.futures import ThreadPoolExecutor, as_completed
from azure.storage.blob import BlobServiceClient
from msticpy.data import QueryProvider

# ----------------------------
# 🔧 CONFIGURATION SECTION
# ----------------------------

# Load environment variables (like secrets)
load_dotenv()

# Azure storage account connection string from .env
STORAGE_CONNECTION_STRING = os.getenv("AZURE_STORAGE_CONNECTION_STRING")

# Define the time range for queries
START_TIME = dt.datetime.utcnow() - dt.timedelta(days=365)
END_TIME = dt.datetime.utcnow()

# List of Log Analytics tables to export
TABLES = ["SecurityEvent", "Syslog", "SigninLogs"]

# How big to start each chunk (can shrink if needed)
INITIAL_CHUNK_HOURS = 1
MIN_CHUNK_MINUTES = 5
MAX_ROWS = 500_000  # hard cap enforced by Log Analytics

# Number of times to retry failed blob uploads
RETRY_LIMIT = 3

# Number of tables to export in parallel
MAX_THREADS = 3

# Where to save the metadata of each export
METADATA_LOG = "export_metadata.csv"

# ----------------------------
# 🧾 LOGGING SETUP
# ----------------------------

logging.basicConfig(level=logging.INFO, format="%(asctime)s [%(levelname)s] %(message)s")
logger = logging.getLogger()

# ----------------------------
# 🔌 DATA SOURCE SETUP (MSTICPy)
# ----------------------------

qry_prov = QueryProvider("AzureMonitor")  # Assumes you are authenticated with Azure

# ----------------------------
# ☁️ STORAGE CLIENT SETUP
# ----------------------------

blob_service_client = BlobServiceClient.from_connection_string(STORAGE_CONNECTION_STRING)

# Collect metadata for final report
metadata_entries = []

# ----------------------------
# 📊 Query a time window of data from a table
# ----------------------------
def run_kql(table, start_time, end_time):
    query = f"{table} | where TimeGenerated between (datetime('{start_time}') .. datetime('{end_time}'))"
    df = qry_prov.exec_query(query)
    return df

# ----------------------------
# 📤 Upload a blob with retry/backoff
# ----------------------------
def upload_blob_with_retry(container_client, blob_name, data, retry_limit=RETRY_LIMIT):
    for attempt in range(1, retry_limit + 1):
        try:
            container_client.upload_blob(blob_name, data, overwrite=True)
            return True
        except Exception as e:
            logger.warning(f"[Attempt {attempt}] Upload failed: {e}")
            time.sleep(2 ** attempt)  # exponential backoff
    return False  # fail after max retries

# ----------------------------
# 🔁 Export all records from a single table
# ----------------------------
def export_table(table):
    logger.info(f"Starting export for table: {table}")
    container_client = blob_service_client.get_container_client(table.lower())
    container_client.create_container(exist_ok=True)  # create if not already there

    current_start = START_TIME
    chunk_hours = INITIAL_CHUNK_HOURS

    while current_start < END_TIME:
        current_end = min(current_start + dt.timedelta(hours=chunk_hours), END_TIME)

        # Initialize metadata entry for this batch
        log_entry = {
            "table": table,
            "start_time": current_start.isoformat(),
            "end_time": current_end.isoformat(),
            "rows": 0,
            "blob_name": None,
            "success": False,
            "duration_sec": 0
        }

        logger.info(f"[{table}] Querying: {current_start} → {current_end} ({chunk_hours}h)")
        t0 = time.time()

        try:
            df = run_kql(table, current_start, current_end)
        except Exception as e:
            logger.error(f"[{table}] Query failed: {e}")
            current_start = current_end
            continue

        duration = time.time() - t0
        log_entry["rows"] = len(df)
        log_entry["duration_sec"] = round(duration, 2)

        # If results hit the cap, reduce chunk size and retry
        if len(df) >= MAX_ROWS:
            if chunk_hours * 60 <= MIN_CHUNK_MINUTES:
                logger.warning(f"[{table}] Chunk too large even at min size. Skipping: {current_start}")
                current_start = current_end
            else:
                chunk_hours /= 2
            continue

        if len(df) > 0:
            # Serialize the data to GZipped JSON
            blob_name = f"{table}_{current_start:%Y%m%dT%H%M}_{current_end:%Y%m%dT%H%M}.json.gz"
            log_entry["blob_name"] = blob_name

            json_bytes = io.BytesIO()
            with gzip.GzipFile(fileobj=json_bytes, mode='w') as f:
                f.write(df.to_json(orient='records', lines=True).encode())
            json_bytes.seek(0)

            # Upload the blob
            success = upload_blob_with_retry(container_client, blob_name, json_bytes)
            log_entry["success"] = success

            if success:
                logger.info(f"[{table}] Uploaded {blob_name}")
            else:
                logger.error(f"[{table}] Upload failed for {blob_name}")

        current_start = current_end
        metadata_entries.append(log_entry)

# ----------------------------
# 🚀 Main loop to run all exports in parallel
# ----------------------------
def main():
    with ThreadPoolExecutor(max_workers=MAX_THREADS) as executor:
        futures = [executor.submit(export_table, table) for table in TABLES]
        for future in as_completed(futures):
            future.result()  # force exceptions to be raised if any

    # Save metadata to CSV
    df_log = pd.DataFrame(metadata_entries)
    df_log.to_csv(METADATA_LOG, index=False)
    logger.info(f"Export metadata saved to: {METADATA_LOG}")

# ----------------------------
# 🔧 Entry point
# ----------------------------
if __name__ == "__main__":
    main()
